# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 200)

import sys
import os
import re
import nltk
from sqlalchemy import create_engine
import pickle
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import gmean

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [2]:
# load data from database
database_filepath = "../data/disaster_response.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name,engine)

### 2. Write a tokenization function to process your text data

In [3]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
#Remove child alone field because it has all zeros only
df = df.drop(['child_alone'],axis=1)

In [5]:
# check the number of 2's in the related field
df['related'].eq(2).sum()

188

In [6]:
# Replace 2 with 1 to consider it a valid response(binary).
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [7]:
# Extract X and y variables from the data for the modelling
X = df['message']
#select from columns with categorical values 0 or 1
y = df.iloc[:,4:]

In [8]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    function to tokenize and normalize text data
    
    Arguments:
        text -> messages to be tokenized and normalized
    Output:
        normalized -> List of tokens extracted and normalized from the messages
    """
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    normalized = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return normalized

In [9]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    The class for implementing Verb Extractor
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline_one = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline_two = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline_one.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5075
               request       0.77      0.57      0.66      1148
                 offer       0.08      0.03      0.04        36
           aid_related       0.76      0.60      0.67      2709
          medical_help       0.62      0.28      0.39       513
      medical_products       0.61      0.32      0.42       314
     search_and_rescue       0.64      0.25      0.36       173
              security       0.23      0.07      0.10       107
              military       0.57      0.31      0.40       203
                 water       0.73      0.66      0.69       420
                  food       0.80      0.68      0.73       742
               shelter       0.77      0.58      0.66       577
              clothing       0.59      0.39      0.47        98
                 money       0.60      0.23      0.33       159
        missing_people       0.52      

In [13]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15019
               request       0.79      0.56      0.65      3326
                 offer       0.47      0.09      0.14        82
           aid_related       0.76      0.60      0.67      8151
          medical_help       0.65      0.28      0.39      1571
      medical_products       0.70      0.36      0.47       999
     search_and_rescue       0.66      0.22      0.33       551
              security       0.52      0.09      0.16       364
              military       0.68      0.37      0.47       657
                 water       0.77      0.67      0.72      1252
                  food       0.82      0.69      0.75      2181
               shelter       0.79      0.54      0.64      1737
              clothing       0.77      0.49      0.60       307
                 money       0.66      0.33      0.44       445
        missing_people       0.69    

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# pipeline_one.get_params().keys()
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

#parameters_grid = {'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline_one, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x136bc8170>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05],
                         'classifier__estima

In [15]:
# Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5075
               request       0.61      0.40      0.48      1148
                 offer       0.00      0.00      0.00        36
           aid_related       0.80      0.19      0.31      2709
          medical_help       0.70      0.12      0.21       513
      medical_products       0.68      0.11      0.20       314
     search_and_rescue       0.60      0.20      0.30       173
              security       0.00      0.00      0.00       107
              military       0.00      0.00      0.00       203
                 water       0.58      0.83      0.68       420
                  food       0.77      0.68      0.72       742
               shelter       0.83      0.29      0.43       577
              clothing       0.64      0.23      0.34        98
                 money       1.00      0.02      0.04       159
        missing_people       0.66      

In [17]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.76      1.00      0.87     15019
               request       0.61      0.37      0.46      3326
                 offer       1.00      0.02      0.05        82
           aid_related       0.80      0.19      0.31      8151
          medical_help       0.61      0.11      0.19      1571
      medical_products       0.74      0.13      0.22       999
     search_and_rescue       0.65      0.19      0.29       551
              security       0.00      0.00      0.00       364
              military       0.50      0.00      0.01       657
                 water       0.58      0.85      0.69      1252
                  food       0.77      0.68      0.72      2181
               shelter       0.85      0.30      0.44      1737
              clothing       0.80      0.34      0.47       307
                 money       1.00      0.00      0.00       445
        missing_people       0.70    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
#Use pipeline_two which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline_two.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5032
               request       0.76      0.55      0.64      1076
                 offer       0.00      0.00      0.00        30
           aid_related       0.75      0.59      0.66      2653
          medical_help       0.63      0.25      0.36       504
      medical_products       0.68      0.34      0.45       313
     search_and_rescue       0.67      0.24      0.36       165
              security       0.37      0.09      0.14       111
              military       0.66      0.35      0.46       224
                 water       0.71      0.61      0.66       417
                  food       0.79      0.69      0.74       745
               shelter       0.79      0.54      0.64       566
              clothing       0.70      0.57      0.63        91
                 money       0.45      0.33      0.38       122
        missing_people       0.54      

In [19]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15062
               request       0.78      0.56      0.65      3398
                 offer       0.42      0.11      0.18        88
           aid_related       0.78      0.61      0.68      8207
          medical_help       0.65      0.28      0.39      1580
      medical_products       0.71      0.35      0.47      1000
     search_and_rescue       0.67      0.20      0.30       559
              security       0.55      0.09      0.15       360
              military       0.68      0.36      0.47       636
                 water       0.78      0.67      0.72      1255
                  food       0.82      0.70      0.75      2178
               shelter       0.80      0.54      0.64      1748
              clothing       0.77      0.48      0.59       314
                 money       0.66      0.34      0.45       482
        missing_people       0.73    

### 9. Export your model as a pickle file

In [20]:
pickled_file = pickle.dumps('../models/classifier.pkl')